TODO:
* add plotting functionality
* add global package importing function
* Either use the guys web scraping code or trey again with
sec-edgar-downloader api
* maybe even try to implement the rest of the project on the data you already
 have, before moving on to downloading an extra data


In [37]:

# import project packages
import wrds
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sec_edgar_downloader import Downloader
import time
import calendar
from bs4 import BeautifulSoup
import os
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pysentiment2 as ps
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
# this function retrieves all sp500 records from the WRDS database
# the records are returned in Pandas dataframe format
def get_sp500_records():
    WRDS_USERNAME = 'denisfench'
    # establish connection to the WRDS database
    conn = wrds.Connection(wrds_username=WRDS_USERNAME)

    ### Get S&P500 Index Membership from CRSP
    sp500 = conn.raw_sql("""
                            select a.*, b.date, b.ret
                            from crsp.msp500list as a,
                            crsp.msf as b
                            where a.permno=b.permno
                            and b.date >= a.start and b.date<= a.ending
                            and b.date>='01/01/2000'
                            order by date;
                            """, date_cols=['start', 'ending', 'date'])

    ### Add Other Company Identifiers from CRSP.MSENAMES, such as TICKER, SHRCD, EXCHCD
    mse = conn.raw_sql("""
                            select comnam, ncusip, namedt, nameendt,
                            permno, shrcd, exchcd, hsiccd, ticker
                            from crsp.msenames
                            """, date_cols=['namedt', 'nameendt'])

    # if nameendt is missing then set to today date
    mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

    # Merge with SP500 data
    sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

    # Impose the date range restrictions
    sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                                & (sp500_full.date<=sp500_full.nameendt)]

    ### Add Other Company Identifiers from CRSP.MSENAMES
    mse = conn.raw_sql("""
                            select comnam, ncusip, namedt, nameendt,
                            permno, shrcd, exchcd, hsiccd, ticker
                            from crsp.msenames
                            """, date_cols=['namedt', 'nameendt'])

    # if nameendt is missing then set to today date
    mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

    # Merge with SP500 data
    sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

    # Impose the date range restrictions
    sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                                & (sp500_full.date<=sp500_full.nameendt)]

    ### Add Compustat Identifiers
    ccm=conn.raw_sql("""
                      select gvkey, liid as iid, lpermno as permno,
                      linktype, linkprim, linkdt, linkenddt
                      from crsp.ccmxpf_linktable
                      where substr(linktype,1,1)='L'
                      and (linkprim ='C' or linkprim='P')
                      """, date_cols=['linkdt', 'linkenddt'])

    # if linkenddt is missing then set to today date
    ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

    # Merge the CCM data with S&P500 data
    # First just link by matching PERMNO
    sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

    # Then set link date bounds
    sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                            &(sp500ccm['date']<=sp500ccm['linkenddt'])]

    # Rearrange columns for final output
    sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', 'linktype', \
                                      'linkprim', 'linkdt', 'linkenddt'])
    sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip',\
                         'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                         'gvkey', 'iid', 'start', 'ending', 'ret']]

    ### Add CIKs and Link with SEC Index Files using CIK
    names = conn.raw_sql(""" select gvkey, cik, sic, naics, gind, gsubind from comp.names """)

    # Merge sp500 constituents table with names table
    sp500 = pd.merge(sp500ccm, names, on='gvkey',  how='left')
    return sp500

In [13]:
# this function retrieves a subsample of the sp500 records corresponding to
# the given dates (YYYY-MM-DD)
# EX: get_sp500_records_by_date(sp500, '2020-01-01', '2021-12-31')

def get_sp500_records_by_date(sp500_record, start_date, end_date):
    new_sp500_sample = sp500_record.loc[start_date <= sp500_record
        .date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

    new_sp500_sample = new_sp500_sample.loc[sp500_record.date <=
                                      end_date][['date',
                                                                        'permno',
                                                          'comnam',
                                                   'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

    return new_sp500_sample

In [64]:
# increment the current date by 1 month
def get_next_month(date, DATE_FORMAT="%Y-%m-%d", datetime_format=False):
    datetime_date = datetime.strptime(date, DATE_FORMAT)
    new_date = datetime_date + relativedelta(months=1)
    if datetime_format:
        return new_date
    else:
        return new_date.strftime(DATE_FORMAT)


# increment the current date by 1 day
def get_next_day(date, datetime_format=False):
    DATE_FORMAT = "%Y-%m-%d"
    datetime_date = datetime.strptime(date, DATE_FORMAT)
    new_date = datetime_date + relativedelta(days=1)
    if datetime_format:
        return new_date
    else:
        return new_date.strftime(DATE_FORMAT)


# get the last day of the current month
def get_end_month_date(curr_date, datetime_format=False):
    DATE_FORMAT = "%Y-%m-%d"
    datetime_date = datetime.strptime(curr_date, DATE_FORMAT)
    current_date = curr_date
    month_end_date = current_date[:8] + str(calendar.monthrange(datetime_date.year,
                                                datetime_date
                                      .month)[1])
    month_end_date = datetime.strptime(month_end_date, DATE_FORMAT)
    if datetime_format:
        return month_end_date
    else:
        return month_end_date.strftime(DATE_FORMAT)

In [82]:
# this function retrieves all 10-K and 10-Q fillings for a given company for
# a given month
def get_filing_by_month(ciks, date, dir_name):
    num_filings = 0
    dl = Downloader(dir_name)
    for cik in ciks:
        num_filings += dl.get("10-Q", cik, after=date, before=get_next_month
        (date))
        num_filings += dl.get("10-K", cik, after=date, before=get_next_month
        (date))
    return num_filings

# this function retrieves all 10-K and 10-Q fillings for a given company for
# a given day
def get_filing_by_day(ciks, date, dir_name):
    num_filings = 0
    dl = Downloader(dir_name)
    for cik in ciks:
        num_filings += dl.get("10-Q", cik, after=date, before=get_next_day
        (date))
        num_filings += dl.get("10-K", cik, after=date, before=get_next_day
        (date))
    return num_filings

In [75]:
# this function retrieves all S&P 500 10-Q or 10-K filings for a given month,
# if such a filing exists for a given company and writes them to a given directory
# the date should be passed in the following format: YYYY-MM-DD
# the company lookup is based in the cik number by default
def get_all_sp_filings_by_month(sp500_collection, date, dir_name,
                                identifier='cik'):
    sp500_ids = []
    num_filings = 0
    DATE_FORMAT = "%Y-%m-%d"
    current_date = date
    month_end_date = get_end_month_date(current_date, datetime_format=True)

    while datetime.strptime(current_date, DATE_FORMAT) <= month_end_date:
        sp500_companies = sp500_collection.loc[sp500_collection.date ==
                                               current_date][['date',
                                                                        'permno',
                                                          'comnam',
                                                   'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

        sp500_ids.extend(sp500_companies[identifier].tolist())
        current_date = get_next_day(current_date)

    num_filings += get_filing_by_month(sp500_ids,
                                       current_date,
                                       dir_name)

    return num_filings

# this function retrieves all S&P 500 10-Q or 10-K filings for a given day,
# if such a filing exists for a given company and writes them to a given directory
# the date should be passed in the following format: YYYY-MM-DD
def get_all_sp_filings_by_day(sp500_collection, date, dir_name, identifier='cik'):
    num_filings = 0
    sp500_companies = sp500_collection.loc[sp500_collection.date == date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]
    sp500_tickers = sp500_companies[identifier]

    for ticker in sp500_tickers:
        num_filings += get_filing_by_day(ticker, date, dir_name)

    return num_filings

In [24]:
# this function retrieves all S&P 500 10-Q and 10-K filings for a given period
def get_sp500_filings(sp500_collection, filings_start_date, filings_end_date,
                      dir_name):

    print("Retrieving all the 10-Q and 10-K filings starting from ",
           filings_start_date, " until ", filings_end_date)

    DATE_FORMAT = "%Y-%m-%d"
    current_date = filings_start_date
    num_filings = 0
    while datetime.strptime(current_date, DATE_FORMAT) <= datetime.strptime\
                (filings_end_date, DATE_FORMAT):
        num_filings += get_all_sp_filings_by_month(sp500_collection, current_date,
                                                   dir_name)
        current_date = get_next_month(current_date)

        print("Successfully retrieved " + str(num_filings) + " for " + str
        (current_date))

    return num_filings

In [28]:
# this function retrieves all tickers for all companies from a given directory
def get_all_filing_tickers(dir_name):
    files = os.listdir(dir_name + "/sec-edgar-filings")
    return files

In [30]:
# this function extracts the filing date, given the filing date line from the
# filing document
def extract_filing_date(filed_as_of_date_line):
    DATE_FORMAT = "%Y%m%d"
    date = ""
    for ch in filed_as_of_date_line:
        if ch.isdigit():
            date += ch
    return datetime.strptime(date, DATE_FORMAT).date()

In [31]:
FILING_DETAILS = "filing-details.html"
FULL_SUBMISSION = "full-submission.txt"
FILED_AS_OF_DATE_IDX = 7

# this function will search for a filing in a given directory based on the
# given ticker
# it will return a dictionary with a ticker and the date of filing as the key
# and filing text as the value
# currently the function doesn't indicate whether the filing is a 10-Q or 10-K
def get_filings_from_ticker(ticker, dir_name, sec_filings):
    for root, dirs, files in os.walk(dir_name + "/sec-edgar-filings/" + ticker):
        for f in files:
            if FILING_DETAILS in f:
                with open(root + "/" + f) as fp:
                    soup = BeautifulSoup(fp, "html.parser")
                    line_idx = 0
                    with open(root + "/" + FULL_SUBMISSION) as fs_fp:
                        while line_idx < FILED_AS_OF_DATE_IDX:
                            line_idx += 1
                            fs_fp.readline()
                        date = extract_filing_date(fs_fp.readline())
                    sec_filings[(ticker, date)] = soup.get_text()
    return sec_filings

In [32]:
import ssl

# this function sets up SSL context for downloading packages from nltk library
def create_ssl_context():
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

In [34]:
# a function that tokenizes 10-K or 10-Q corpus that
def tokenize_filing(filing_corpus):
    filing_tokenizer = RegexpTokenizer(r'\w+')
    return filing_tokenizer.tokenize(filing_corpus)

In [36]:
# a function to remove the stop words from the filings corpus
def filter_out_stopwords(tokenized_filing_corpus):
    stop_words = set(stopwords.words('english'))
    filing_corpus_filtered = [word for word in tokenized_filing_corpus if not \
        word.lower() in stop_words]
    return filing_corpus_filtered


# a function that filters out numbers from a filing corpus
def filter_out_numbers(filing_corpus):
    return [token for token in filing_corpus if not (token.isdigit()
                                         or token[0] == '-' and token[1:].isdigit())]

In [38]:
# Perform sentiment analysis on 10-Q and 10-K filing documents

# this function calculates the sentiment score of the text based on a
# sentiment dictionary (Loughran or Harvard), passed in as a parameter
def get_sentiment_score(text, dictionary):

    # get the Harvard general sentiment dictionary
    hiv4 = ps.HIV4()

    # get the Loughran and McDonald dictionary
    lm = ps.LM()

    if dictionary == "Harvard":
        return hiv4.get_sentiment_score(text)

    if dictionary == "Loughran":
        return lm.get_sentiment_score(text)

    raise ValueError("The dictionary could not be found.")

In [39]:
# this function retrieves Value-Weighted Return (including distributions) and
# Holding Period Return from the CRSP database
def get_crsp_data():
    CRSP_DATA_DIR = "CRSP-data/"
    crsp_df = pd.read_csv(CRSP_DATA_DIR + "crsp.csv", parse_dates=['date'])
    return crsp_df

In [40]:
# this function calculates excess return using Value-Weighted Return (including distributions) and
# Holding Period Return from the CRSP database
def get_excess_return(crsp_dataframe, permno, date):
    excess_return_df = crsp_dataframe[crsp_dataframe['PERMNO'] == permno]
    excess_return_df =  excess_return_df[crsp_dataframe['date'] == date]
    excess_return = float(excess_return_df['vwretd']) - \
                    float(excess_return_df['RET'])
    return excess_return

In [41]:
DICTIONARY_DIR = "dictionaries/"

# this function constructs a sentiment dictionary based on the Loughran
# McDonald and Harvard dictionaries
# return: {Term: [str], Sentiment: [int], Term_Count: [int]}
# where Term count is the number of times the term occurs within the corpus
def construct_sentiment_dict(base_dict="Loughran"):
    if base_dict == "Loughran":
        df = pd.read_csv(DICTIONARY_DIR +
                         "Loughran-McDonald_MasterDictionary_1993-2021.csv",
                         header=0, usecols=['Word', 'Negative', 'Positive'])
    else:
        df = pd.read_csv(DICTIONARY_DIR + "Harvard.csv", header=0, usecols=['Word', 'Negative', 'Positive'])

    df = df[(df['Negative'] != 0) | (df['Positive'] != 0)]
    return df

In [42]:
# constructing Loughran sentiment dictionary
lr_dict = construct_sentiment_dict()
lr_dict.head()

,Word,Negative,Positive
9,ABANDON,2009,0
10,ABANDONED,2009,0
11,ABANDONING,2009,0
12,ABANDONMENT,2009,0
13,ABANDONMENTS,2009,0


In [43]:
# This function constructs a term dictionary based on a given document: tokenized filing list
# return: {Term: [str], Term_Count: [int]}
# where Term count is the number of times the term occurs within the document
def construct_term_dict(document):
    doc_dict = {}
    for token in document:
        if token in doc_dict:
            doc_dict[token] += 1
        else:
            doc_dict[token] = 1
    return doc_dict

In [44]:
# This function calculates a document negativity score based on the
# proportion of negative words in the document
# the negativity of the term is determined based on a given dictionary
# (Loughran or Harvard)
def compute_negativity_score_based_on_proportion(text,
                                                 sentiment_dict="Loughran"):
    sentiment_score = get_sentiment_score(text, sentiment_dict)
    num_negative = sentiment_score['Negative']
    return num_negative / len(text)

In [45]:
# *** TO BE REMOVED ***

# This function calculates a document negativity score based on the
# TFIDF score for the document
# TFIDF:
# TF (Term Frequency) (t, d) = (# Occurrences of the word in a document / # Total words in a document )
# IDF (Inverse Document Frequency) (t, D) = log ( (N (number of documents in a collection) / (# Documents containing the term t))
# TFIDF (t, d, D) = TF (t, d) * IDF (t, D)
# the function takes document dictionary, containing terms and their
# frequencies for a given document; the sentiment dictionary, either Loughran
# or Harvard, containing token sentiment rating as well as the document term
# count for the entire corpus; the number of documents in the collection and
# the number of tokens within a given document
def get_tfidf_negativity_score(doc_dict, sentiment_dict, doc_term_freq,
                               collection_size):
    pass

In [46]:
# This function calculates TFIDF score for the filings based on the sentiment
# dictionary vocabulary using SciKit Learn
def calc_tfidf_score(sentiment_dict, corpus):
    vectorizer = TfidfVectorizer(vocabulary=sentiment_dict.keys(),
                                 stop_words="english")
    vectors = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    dense_list = dense.tolist()
    tfidf_df = pd.DataFrame(dense_list, columns=feature_names)
    return tfidf_df

In [48]:
# this function loads all the filings from a given directory, which
# constitute an entire corpus of documents
def load_filings_corpus(corpus_root_dir):
    FILING_DETAILS = "filing-details.html"
    sec_filings_corpus = []
    for root, dirs, files in os.walk(corpus_root_dir + "/sec-edgar-filings/"):
        for f in files:
            if FILING_DETAILS in f:
                with open(root + "/" + f) as file_ptr:
                    bs = BeautifulSoup(file_ptr, "html.parser")
                    sec_filings_corpus.append(bs.get_text())
    return sec_filings_corpus

<h1>SENTIMENT ANALYSIS OF 10-Qs and 10-Ks for S&P 500 companies for 2020,
2021<h1>

In [49]:
# getting all S&P 500 records from the WRDS database

sp500_all = get_sp500_records()

Loading library list...
Done


In [54]:
# retrieving all S&P 500 records for the two-year period (from '2020-01-01'
# to '2021-12-31')

sp500_sample = get_sp500_records_by_date(sp500_all, "2020-01-01", "2021-12-31")
sp500_sample.head()

,date,permno,comnam,ncusip,gvkey,iid,cik,ticker,sic,naics
120131,2020-01-31,34746.0,FIFTH THIRD BANCORP,31677310,004640,01,0000035527,FITB,6020,522110
120132,2020-01-31,75591.0,IDEX CORP,45167R10,015267,01,0000832101,IEX,3561,333914
120133,2020-01-31,14541.0,CHEVRON CORP NEW,16676410,002991,01,0000093410,CVX,2911,324110
120134,2020-01-31,14593.0,APPLE INC,03783310,001690,01,0000320193,AAPL,3663,334220
120135,2020-01-31,75341.0,DUKE REALTY CORP,26441150,013510,01,0000783280,DRE,6798,531120


In [ ]:
# get a sample of S&P 500 filings for a 2-year period
start_date = "2020-01-01"
end_date = "2021-12-31"
sp_filings_dir_name = 'sp500_filings_sample'
num_filings_sample = get_sp500_filings(sp500_sample, start_date, end_date, sp_filings_dir_name)

Retrieving all the 10-Q and 10-K filings starting from  2020-01-01  until  2021-12-31


/Users/dennisfenchenko/NYU-Fall-2022/nlp-finance/10k-sentiment-analysis/venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
